In [4]:
import re
class Stack:
    """Stack class"""
    def __init__(self, capacity):
        self.capacity = capacity
        self.items = []

    def is_empty(self):
        return self.items == []

    def is_full(self):
        return len(self.items) == self.capacity

    def push(self, item):
        if self.is_full():
            raise IndexError()
        else:
            self.items.append(item)

    def pop(self):
        if self.is_empty():
            raise IndexError()
        else:
            temp = self.items[-1]
            self.items = self.items[0:-1]
            return temp

    def peek(self):
        if self.is_empty():
            raise IndexError()
        else:
            return self.items[-1]

    def size(self):
        return len(self.items)

In [5]:
def infix_to_postfix(input_str):
    """Takes in an expression written in the common infix notation and transforms it into an expression
    in the postix notation"""
    precedence = {'(': 1, '-': 2, '+': 2, '/': 3, '*': 3, '^': 4}
    op_stack = Stack(30)
    
    postfix_list = []
    token_list = input_str.split()
    
    for token in token_list:
        if ");" in token:
            token = token.replace(");", "")
        
        elif ";" in token:
            token = token.replace(";", "")
        
        elif "(" in token and "()" not in token:
            token = token.replace("(", "")
        
        elif ")" in token and "()" not in token:
            token = token.replace(")", "")
        
        if token not in "^*/+-()":
            postfix_list.append(token)
        elif token == '(':
            op_stack.push(token)
        elif token == ')':
            while op_stack.peek() != '(':
                postfix_list.append(op_stack.pop())
            op_stack.pop()
        else:
            prec = precedence[token]
            while (not op_stack.is_empty()) and \
                  ((token != "^" and precedence[op_stack.peek()] >= prec) or (precedence[op_stack.peek()] > prec)):
                postfix_list.append(op_stack.pop())
            op_stack.push(token)
    
    while not op_stack.is_empty():
        postfix_list.append(op_stack.pop())
    return ' '.join(postfix_list)



In [7]:
infix_to_postfix("(1 + 2) + 7 * 111")

'1 2 + 7 111 * +'

In [239]:
class Lexer():
    def __init__(self, variables, operators, signs, calls, defs):
        """Lexer class; takes in symbol tables as an input"""
        self.variables = variables
        self.operators = operators
        self.signs = signs
        self.calls = calls
        self.defs = defs
        
        self.stack = []
        self.tokens = []
        self.pos = []
        self.poscode = []
        
    def infix_pos(self, inp):
        """Convert each line from infix to posfix"""
        pos = infix_to_postfix(inp)
        self.break_down(pos, self.pos)       
    
    def getCode(self, inp):
        self.break_down(inp, self.stack)
    
    def break_down(self, inp, out):
        """Break Down the Input"""
        result = []
        temp = ""
        for char in inp:
            #if character is a number or a character from alphabet and not a sign 
            if char != " " and re.match('^[a-zA-Z0-9_]+$', char) and char not in self.operators \
            and char not in self.signs and char not in self.calls:
                temp = temp + char
            #if the character is a sign
            elif char != " " and (char in self.operators or char in self.signs):
                result.append(temp)  #append the word
                result.append(char)  #append the sign
                temp = ""            #clear the word
            elif char == " ":       
                result.append(temp)  #append the word when the current char is blank
                temp = ""
        
        if temp != "":      
            result.append(temp)
            
        out.append(result)
            
    def pos_code(self):
        """Method for cleaning up the posfix code"""
        c = []
        
        for line in self.pos:           #loop through and remove any blanks
            for var in line:
                if var in self.variables:
                    c.append(var)
                elif var in self.operators:
                    c.append(var)
                elif var in self.signs:
                    c.append(var)
                elif var in self.calls:
                    c.append(var)    
                elif var in self.defs:
                    c.append(var)    
                elif re.match('^[a-zA-Z_]+$', var):
                    c.append(var)
                elif re.match('^[0-9]+$', var):
                    c.append(var)
        
            if not("def" in c) or ("}" in c):
                self.poscode.append(c)
                c = []
    
    def tokenize(self):
        """Tokenize each line of code"""
        t = []
        
        for line in self.stack:
            for var in line:
                if var in self.variables:
                    t.append(self.variables[var])
                   
                elif var in self.operators:
                    t.append(self.operators[var])
                   
                elif var in self.signs:
                    t.append(self.signs[var])
                
                elif var in self.calls:
                    t.append(self.calls[var])
                
                elif var in self.defs:
                    t.append(self.defs[var])
                
                elif re.match('^[a-zA-Z_]+$', var):
                    t.append("NAME")
                
                elif re.match('^[0-9]+$', var):
                    t.append("OPERAND")
                
                
            if not("DEFINE" in t) or ("CLOSED-B" in t):
                self.tokens.append(t)
                t = []
    

In [240]:
class DictList(dict):
    """Special Dictionary To store duplicate Key's items inside a list"""
    def __setitem__(self, key, value):
        try:
            # Assumes there is a list on the key
            self[key].append(value) 
        except KeyError: # if fails because there is no key
            super(DictList, self).__setitem__(key, value)
        except AttributeError: # if fails because it is not a list
            super(DictList, self).__setitem__(key, [self[key], value])
        
class State():
    def __init__(self, name):
        """Class for State"""
        self.name = name
        self.dest = DictList()
        self.is_end = False   #variable checking if the state has ended
        
    def getName(self):
        return self.name
    
    def addTrans(self, char, state):
        """With the usage of the custom Dictionary, it's easier to add new items"""
        self.dest[char] = state
       
        

In [241]:
class Parser():
    def __init__(self):
        """Parser class; responsible for checking syntax rules of the code"""
        #create the states
        var = State("VAR")
        name = State("NAME")
        equal = State("EQUAL")
        operand = State("OPERAND")
        operator = State("OPERATOR")
        define = State("DEFINE")
        open_p = State("OPEN-P")
        closed_p = State("CLOSED-P")
        open_b = State("OPEN-B")
        closed_b = State("CLOSED-B")
        ret = State("RETURN")
        outp = State("PRINT")
        semi_colon = State("BREAK")
        comma = State("COMMA")
        
        #declare end state
        semi_colon.is_end = True
        closed_b.is_end = True
        
        #add transition
        var.addTrans("NAME", name)
        
        name.addTrans("EQUAL", equal)
        name.addTrans("OPERATOR", operator)
        name.addTrans("BREAK", semi_colon)
        name.addTrans("OPEN-P", open_p)
        name.addTrans("COMMA", comma)
        name.addTrans("CLOSED-P", closed_p)
        
        equal.addTrans("OPERAND", operator)
        equal.addTrans("NAME", operator)

        operand.addTrans("OPERATOR", operator)
        operand.addTrans("BREAK", semi_colon)

        operator.addTrans("OPERAND", operand)
        operator.addTrans("NAME", name)
        
        define.addTrans("NAME", name)
        
        open_p.addTrans("VAR", var)
        open_p.addTrans("RETURN", ret)
        open_p.addTrans("CLOSED-B", closed_b)
     
        closed_p.addTrans("BREAK", semi_colon)
        closed_p.addTrans("OPEN-B", open_b)
        
        open_b.addTrans("VAR", var)
        open_b.addTrans("RETURN", ret)
        
        ret.addTrans("NAME", name)
        ret.addTrans("OPERAND", operand)
        
        outp.addTrans("NAME", name)
        outp.addTrans("OPERAND", operand)
        
        
        comma.addTrans("VAR", var)
        comma.addTrans("NAME", name)
        
        semi_colon.addTrans("CLOSED-B", closed_b)
        semi_colon.addTrans("VAR", var)
        semi_colon.addTrans("DEFINE", define)
        
        #add all of the states into a dictionary
        #key value is the input and item is the state
        self.statesL = {"VAR": var, "NAME": name, "OPERAND": operand,
                  "EQUAL" : equal, "OPERATOR" : operator, "BREAK" : semi_colon,
                  "DEFINE" : define, "OPEN-P" : open_p, "CLOSED-P" : closed_p,
                  "OPEN-B" : open_b, "CLOSED-B" : closed_b, "RETURN" : ret,
                  "COMMA" : comma, "PRINT" : outp }
    
    def check(self, tokens):
        """Check method use for validating the lines of code using the tokens"""
        current = None
        stateS = []
        for line in tokens:         #loop through each line
            for token in line:      #loop through each token in the line
                if current == None:      
                    try:
                        current = self.statesL[token]
                    except:
                        print("INVALID")
                else:
                    if token in current.dest:       
                        current = current.dest[token]     #set current to the destination
            
            if current.is_end == True:  
                #if the valid end state is true,
                stateS.append(True)
            else:
                stateS.append(False)
                
            current = None        #reset current
    
        if False not in stateS:
            return True
        return False

In [242]:
class Intermediate():
    def __init__(self, variables=None, operators=None, signs=None, calls=None, defs=None, code=None):
        """Intermediate Generator; Convert the source code into Assembly code while optimizing the
            the code by properly arranging the sequence of instructions
        """
        #takes in the symbol tables and the broken down code from the compiler as inputs 
        self.code = code
        self.variables = variables
        self.operators = operators
        self.signs = signs
        self.calls = calls
        self.defs = defs
        
        #a table for operation comparison
        self.operations = {"+" : "ADD", "-": "SUB"}
        
        self.intermediateCode = []
        self.functions = {}    #a dictionary functions; uses to takes in function name and its operation stack
        self.operands = []     #a global stack for storing operands of global statements
        self.operators = []    #a global stack for storing operator of global statements
        
        self.names = []        #a stack that takes in the names of the variables/functions
        self.localN = []
        
    def searchS(self, start, line, intermediate):
        """A Method for stripping down the statement and grab only operands and operators"""
        temp = line[start:]  #strip the list and only grab the operands and operators
        temp.reverse()       #reverse the stack for proper pops
        self.generateS(temp, intermediate)      #call the method generateS to generate intermediate code
    
    def generateS(self, s1, intermediate):
        """Generate intermediate code using two stacks and interchange between them"""   
        s2 = []       #temporary stack
        while (len(s1) != 0):
            pop = s1.pop()     #pop s1 until its empty
            s2.append(pop)     #append to s2
            
            if len(s2) >= 3 and s2[-1] in self.operations:  #if the top of s2 is an operation
                operator = s2.pop()            #pop three times if len > 3
                op1 = s2.pop()
                op2 = s2.pop()
                
                intermediate.append(("LOAD",op2))
                intermediate.append(("LOAD",op1))
                intermediate.append(self.operations[operator])  
            
            elif len(s2) >= 2 and s2[-1] in self.operations:
                operator = s2.pop()       #pop three times if len > 2
                op1 = s2.pop()
                intermediate.append(("LOAD",op1))
                intermediate.append(self.operations[operator])
            
            elif len(s2) == 2 and s2[-1] in self.calls:       #if the top of s2 is a RETURN
                operator = s2.pop()        #pop twice
                op1 = s2.pop()
                intermediate.append((self.operations[operator],str(op1)))
                
    def stripF(self, line):
        """A method for breaking down functions"""
        stack = []
        temp = []
        
        for index in range(len(line)): #scan the function line and grab only the operannds/operators
            if (line[index] not in self.defs) and \
            (line[index] == "=" or line[index] in self.variables \
            or line[index] in self.operations or re.match('^[a-zA-Z0-9_]+$', line[index])) \
            and (line[index + 1] in self.variables or line[index + 1] == "return"):
                temp.append(line[index])
                stack.append(temp)
                temp = []
            
            elif (line[index] not in self.defs) and \
            (line[index] == "=" or line[index] in self.variables \
            or line[index] in self.operations or re.match('^[a-zA-Z0-9_]+$', line[index])) \
            and line[index + 1] not in self.variables:
                temp.append(line[index])
        
        if len(temp) != 0:
            stack.append(temp)
            
        return stack
    
    def searchF(self, start, line):
        line = self.stripF(line[start:])
        return line
                
    def search(self):
        for line in self.code:
            if line[0] in self.variables and line[2] in self.signs: #int name = 
                self.names.append(line[1])
                self.searchS(3, line, self.intermediateCode)
                self.intermediateCode.append(("PUSH",line[1]))
              
            elif line[0] not in self.variables and line[1] in self.signs : #name =
                self.names.append(line[0])
                self.searchS(2, line, self.intermediateCode)
                self.intermediateCode.append(("PUSH",line[0]))
            
            elif line[0] == "print" and len(line) > 3:
                self.searchS(1, line, self.intermediateCode)
                self.intermediateCode.append("PRINT")
                        
            elif line[0] == "print" and len(line) == 2:
                self.intermediateCode.append(("PRINT", line[1]))
                
            elif line[0] in self.defs: #def name
                self.names.append(line[1])
                self.intermediateCode.append(("PUSH",line[1]))
                stack = self.searchF(3, line)
                temp = []
                intermediate = []
            
                for lineS in stack:
                    if lineS[0] in self.variables and lineS[2] in self.signs:
                        self.searchS(3, lineS, intermediate)
                        intermediate.append(("PUSH", lineS[1]))
                        self.localN.append(lineS[1])
                    elif lineS[0] not in self.variables and lineS[1] in self.signs:
                        self.searchS(2, lineS, intermediate)
                        intermediate.append(("PUSH", lineS[0]))
                        self.localN.append(lineS[0])
                    elif lineS[0] == "return" and len(lineS) > 3:
                        self.searchS(1, lineS, intermediate)
                        intermediate.append("RETURN")
            
                    elif lineS[0] == "return" and len(lineS) == 2:
                        intermediate.append(("RETURN", lineS[1]))
                    
                    elif lineS[0] == "print" and len(lineS) > 3:
                        self.searchS(1, lineS, intermediate)
                        intermediate.append("PRINT")
                        
                    elif lineS[0] == "print" and len(lineS) == 2:
                        intermediate.append(("PRINT", lineS[1]))
                
                self.functions[line[1]] = intermediate
        
        self.localN.reverse()
        self.names.reverse()
        print(*self.intermediateCode, sep='\n')
        print(self.functions)

In [243]:
class Linker():
    def __init__(self, functions, names, localN):
        self.globalN = names
        self.localN = localN
        self.funct_dict = functions
    
        self.operations = {'LOAD-OP' : '000', 'LOAD-G': '001', 'LOAD-L': '010', 
                          'ADD' : '011', 'SUB' : '100', 'PRINT' : '101',
                          'PUSH' : '110', 'RETURN': '111', 'LOAD': ''}
        
        self.MC = []
        
        self.counter = 0
        self.loadC = 0
        
        self.dictG = {}
        self.dictL = {}
        
    def load(self, line):
        try:
            self.MC.append(self.operations['LOAD-OP'] + '{0:013b}'.format(int(line[1])))
            self.counter += 1
            self.loadC += 1
            
        except:
            if line[1] in self.dictG:
                self.MC.append(self.operations['LOAD-G'] + \
                               '{0:013b}'.format(list(self.dictG.keys()).index(line[1])))
                self.counter += 1
                self.loadC += 1
                
            elif line[1] in self.dictL:
                self.MC.append(self.operations['LOAD-L'] + \
                               '{0:013b}'.format(list(self.dictL.keys()).index(line[1])))
                self.counter += 1
                self.loadC += 1
                
    def pushG(self, line):
        temp = self.globalN.pop()
        self.dictG[temp] = None
        self.MC.append(self.operations[line[0]] + '0' + '{0:012b}'.format(list(self.dictG.keys()).index(temp)))
        self.counter += 1
    
    def pushF(self, line):
        temp = self.localN.pop()
        self.dictL[temp] = None
        self.MC.append(self.operations[line[0]] + '1' + '{0:012b}'.format(list(self.dictL.keys()).index(temp)))
        self.counter += 1
    
    def genMCF(self, line, code):
        for line in code:
            if type(line) == tuple and line[0] in self.operations:
                if line[0] == 'LOAD':
                    self.load(line)
                elif line[0] == 'PUSH':
                    self.pushF(line)
                elif (line[0] == 'RETURN' or line[0] == 'PRINT'):
                    self.genReturnSet(line)
            
            elif type(line) == str and line in self.operations:
                if line != 'RETURN' and line != 'PRINT':
                    self.genArith(line)
                else:
                    #if print
                    self.genReturn(line) 
    
    def genArith(self, line):
        if self.loadC == 2:
            self.MC.append(self.operations[line] + '0' + \
                           '{0:06b}'.format(self.counter-2) + '{0:06b}'.format(self.counter-1))
            self.loadC = 0
            self.counter += 1
            
        elif self.loadC == 1:
            self.MC.append(self.operations[line] + '1' + \
                           '{0:06b}'.format(0) + '{0:06b}'.format(self.counter-1))
            self.loadC = 0
            self.counter += 1
            
    def genReturn(self, line):
        if self.loadC == 1:
            #return 1 operand
            self.MC.append(self.operations[line] + '00' + '{0:06b}'.format(self.counter-1) + '{0:05b}'.format(0))  
            self.loadC = 0
            self.counter += 1
        elif self.loadC == 2:
            #return 2 operands
            self.MC.append(self.operations[line] + '01' + '{0:05b}'.format(self.counter-2) + '{0:06b}'.format(self.counter-1))
            self.loadC = 0
            self.counter += 1
        
        elif self.loadC == 0:
            #return a value after an arithmetic
            self.MC.append(self.operations[line] + '11' + '{0:05b}'.format(0) + '{0:06b}'.format(0))
            self.loadC = 0
            self.counter += 1
    
    def genReturnSet(self, line):
        try:
            self.MC.append(self.operations[line[0]] + '{0:013b}'.format(int(line[1])))
            self.counter += 1
        except:
            if line[1] in self.dictG:
                self.MC.append(self.operations[line[0]] + '11' + '{0:011b}'.format(list(self.dictG.keys()).index(line[1])))
                self.counter += 1
            elif line[1] in self.dictL:
                self.MC.append(self.operations[line[0]] + '10' + '{0:011b}'.format(list(self.dictL.keys()).index(line[1])))
                self.counter += 1
                
    def gen_mc(self, code):
        for line in code:
            if type(line) == tuple and line[0] in self.operations:
                if line[0] == 'LOAD':
                    self.load(line)
                elif line[0] == 'PUSH' and line[1] not in self.funct_dict:
                    self.pushG(line)
                elif line[0] == 'PUSH' and line[1] in self.funct_dict:
                    self.genMCF(line, self.funct_dict[line[1]])
                    
                elif (line[0] == 'RETURN' or line[0] == 'PRINT'):
                    self.genReturnSet(line)
            
            elif type(line) == str and line in self.operations:
                if line != 'RETURN' and line != 'PRINT':
                    self.genArith(line)
                else:
                    #if print
                    self.genReturn(line)
        
        return self.MC, self.dictG, self.dictL
        

In [244]:
class Compiler():
    def __init__(self):
        """Compiler class"""
        """Defined all of the symbol tables here"""
        
        self.variables = { "int" : "VAR", "float" : "VAR", "string" : "VAR" }

        self.signs = { "=":"EQUAL", "," : "COMMA", "(" : "OPEN-P", ")" : "CLOSED-P",
         ";" : "BREAK", "{": "OPEN-B", "}" : "CLOSED-B"}

        self.define = {"def" : "DEFINE"}
        self.calls = {"return" : "RETURN", "print" : "PRINT"}
        self.operators = { "+" : "OPERATOR", "-": "OPERATOR"}
        
        self.lexer = Lexer(self.variables, self.operators, self.signs, self.calls, self.define)
        self.parser = Parser()
        self.intermediate = Intermediate()
        
    def load(self, file):
        """Load the source file and call the lexer to scan each line"""
        with open(file, 'r') as target:
            for line in target:
                line = line.strip()
                if line:
                    self.lexer.getCode(line)
                    self.lexer.infix_pos(line)
    
    def lex(self):
        """A function that tokenizes the input as well as grabbing the actual code"""
        self.lexer.tokenize()
        self.lexer.pos_code()
        
    def setIntermediate(self, var, op, sign, call, define, code):
        """A method for passing inputs to the Intermediate Code Generation"""
        self.intermediate = Intermediate(var, op, sign, call, define, code)
        self.intermediate.search() #call the method search()
        
    def syntax_check(self):
        """Check the code syntax by using the tokens"""
        return self.parser.check(self.lexer.tokens)
     
    def intermediate_gen(self):
        if self.syntax_check():  #if the syntax is true, check the meaning
            self.setIntermediate(self.variables, self.operators, self.signs, self.calls, self.define, self.lexer.poscode)     
            linker = Linker(self.intermediate.functions, self.intermediate.names, self.intermediate.localN)
            return linker.gen_mc(self.intermediate.intermediateCode)
        return None

In [256]:
compiler = Compiler()
compiler.load("source_code.txt")
compiler.lex()

In [257]:
MC, dictG, dictL = compiler.intermediate_gen()
MC, dictG, dictL

('LOAD', '9')
('LOAD', '7')
ADD
('LOAD', '24')
SUB
('LOAD', '14')
SUB
('PUSH', 'a')
('LOAD', '9')
('LOAD', '2')
ADD
('PUSH', 'a')
('LOAD', '7')
('LOAD', '24')
SUB
('PUSH', 'z')
('LOAD', 'a')
('LOAD', '100')
ADD
('PUSH', 'b')
('LOAD', 'a')
('LOAD', 'b')
ADD
('PUSH', 'c')
('LOAD', 'b')
('LOAD', 'c')
ADD
('PUSH', 'a')
('PRINT', 'a')
('LOAD', '7')
('LOAD', '24')
ADD
PRINT
('PUSH', 'hello')
('PUSH', 'main')
{'hello': [('PRINT', '100'), ('RETURN', '100')], 'main': [('LOAD', 'a'), ('LOAD', '100'), 'SUB', ('PUSH', 'f'), ('LOAD', '150'), ('LOAD', '10'), 'SUB', ('PUSH', 'd'), ('LOAD', 'f'), ('LOAD', 'd'), 'ADD', 'RETURN']}


(['0000000000001001',
  '0000000000000111',
  '0110000000000001',
  '0000000000011000',
  '1001000000000011',
  '0000000000001110',
  '1001000000000101',
  '1100000000000000',
  '0000000000001001',
  '0000000000000010',
  '0110001000001001',
  '1100000000000000',
  '0000000000000111',
  '0000000000011000',
  '1000001100001101',
  '1100000000000001',
  '0010000000000000',
  '0000000001100100',
  '0110010000010001',
  '1100000000000010',
  '0010000000000000',
  '0010000000000010',
  '0110010100010101',
  '1100000000000011',
  '0010000000000010',
  '0010000000000011',
  '0110011000011001',
  '1100000000000000',
  '1011100000000000',
  '0000000000000111',
  '0000000000011000',
  '0110011101011110',
  '1011100000000000',
  '1010000001100100',
  '1110000001100100',
  '0010000000000000',
  '0000000001100100',
  '1000100011100100',
  '1101000000000000',
  '0000000010010110',
  '0000000000001010',
  '1000100111101000',
  '1101000000000001',
  '0100000000000000',
  '0100000000000001',
  '0110101

In [258]:
import time 

class ALU():
    """Arithmetic Logic Unit Class"""
    def __init__(self):
        #a dictionary that holds operations keys and the corresponding operation in accordance to the key
        #Lambda is used to created an empty function in which I assigned it to the corresponding operation
        self.operations = {
            "011" : lambda a, b: self.add(a,b),          #opcode is 01 then it is addition
            "100" : lambda a, b: self.subtract(a,b)      #opcode is 10 then it is subtraction
        }
    
    def add(self, a, b):
        total = int(a, 2) + int(b, 2)
        if total > 2048:
            return '1' + '{0:011b}'.format(total), "True"
        else:
            return '{0:012b}'.format(total), "False"
        
    def subtract(self, a, b):
        total = int(a, 2) - int(b, 2)
        if total < 0:
            total = total * -1
            return '1' + '{0:011b}'.format(total), "Negative"
        else:
            return '0' + '{0:011b}'.format(total), "Positive"
            
    def operate(self, op, op1 , op2):
        """Operate Method"""
        if op in self.operations:      #no need for loop here; this saves some run time complexity
            result, overflow = self.operations[op](op1, op2)
            if op == "01":
                return "ADDITION", result, overflow
            
            else:
                return "SUBTRACTION", result, overflow
        else:
            print("ERROR: Operation not found")
            

In [265]:
class PU():
    def __init__(self, PC, dictG, dictL, AC, RAM, size, n):
        """Processing Unit; takes parameter PC, AC, RAM, size of the RAM;"""
        self.n = n
        self.PC = PC             #PC 
        self.RAM = RAM           #RAM 
        self.length = size       #size of RAM  
        self.MDR = None          #Memory Data Register
        self.CIR = None          #Current Instruction Register
        self.MAR = None          #Memory Address Register
        self.AC = AC             #Accumulator
        self.ALU = ALU()         #ALU
        self.oldPC = 0           #A register that temporarily store the current PC 
        self.calC = 0        #A register that indicates if the instruction is to calculate
 
        self.dictG = dictG     #global dictionary
        self.dictL = dictL     #local dictionary
        
        self.loadGC = 0       #load global check
        self.loadLC = 0       #load local check
        
        self.retC = 0         #return check
        self.printC = 0       #print check
        
        self.pushC = 0        #push check
        
        self.index1 = 0       #index
        self.index2 = 0       #second index  
        
        self.pushS = None         #push scope
        self.target = None        #target 
        
    def loadPC(self, PC):
        return self.RAM[PC]          #return PC contents
    
    def loadOP(self, MAR):
        """LOAD OPERAND"""
        return self.MAR[4:]
    
    def loadCIR(self, MAR):
        return MAR[:3]          #return opcode
    
    def loadG(self, MAR):
        """LOAD GLOBAL VARIABLE"""
        key = int(MAR[4:], 2)
        target = list(self.dictG.keys())[key] #grab the name of the variable
        self.loadGC = 1
        
        print("LOAD GLOBAL VAR", target)
    
        return target
    
    def loadL(self, MAR):
        """LOAD LOCAL VARIABLE"""
        key = int(MAR[4:], 2)
        target = list(self.dictL.keys())[key] #grab the name of the variable
        self.loadLC = 1
        
        print("LOAD LOCAL VAR", target)
        
        return target
    
    def ret(self, MAR):
        check = MAR[3] + MAR[4]
        if check == "00": #return one operand
            if MAR[11:] == "00000":  #checking if the last 5 bits are all zeroes
                 return MAR[5:11], None   #return the index of the actual operand
            else:
                 return MAR[5:], None
                
        elif check == "01": #return two operands
            return MAR[5:11], MAR[11:]
        
        elif check == "11": #this can be returning a global variable or a value after calculation
            if MAR[4:] == "00000000000":   #check if the bits indicate returning a calculated value
                return None, None
            else:          #else return the proper index of the global variable
                return MAR[5:11], "0"     #return index and a 0 bit to indicate global
        
        elif check == "10":  #returning a local variable
            return MAR[5:], "1"
            
    def push(self, MAR):
        check = MAR[3]   #the fourth value in the address indicates whether if the push is local/global
        target = None
        if check == '0':
            """Global PUSH"""
            print("GLOBAL PUSH")
            key = int(MAR[4:], 2)
            target = list(self.dictG.keys())[key] #grab the name of the variable
            print("THIS IS KEY", key)
            print("THIS IS TARGET", target)
            
        elif check == '1':
            """LOCAL PUSH"""
            print("LOCAL PUSH")
            key = int(MAR[4:], 2)
            target = list(self.dictL.keys())[key] #grab the name of the variable
            print("THIS IS KEY", key)
            print("THIS IS TARGET", target)
        
        self.pushC = 1
        return target, check
    
    def fetch(self):
        """Fetching Method"""
        self.MAR = self.loadPC(self.PC)          #load PC contents to MAR
        self.CIR = self.loadCIR(self.MAR)        #load opcode into CIR 
        if self.PC < self.length:
            print("MAR: " + self.MAR + " Instruction: " + self.CIR + " PC: ", self.PC)
            self.PC = self.PC + self.n                    #increment PC
        
    def decode(self):
        if (self.CIR == "000"): #load OPERAND
            if self.MDR == None:
                self.MDR = self.loadOP(self.MAR)
                print("This is MDR", self.MDR)
            else:
                self.AC = self.MDR
                self.MDR = self.loadOP(self.MAR)
                print("MOVE CONTENT TO AC", self.AC)
                print("NEW MDR", self.MDR)
                
        elif (self.CIR == "001"): #load global variable
            self.target = self.loadG(self.MAR)
        
        elif (self.CIR == "010"): #load local variable
            self.target = self.loadL(self.MAR)
        
        elif (self.CIR == "110"): #push instruction
            self.target, self.pushS = self.push(self.MAR)
        
        elif (self.CIR == "100" or self.CIR == "011"):
            self.calC = 1            #if instruction is add or subtract, calCheck is True
            
        elif (self.CIR == "111"): #return 
            self.index1, self.index2 = self.ret(self.MAR)
            print("INDEX", self.index1 + " INDEX2", self.index2)
            self.retC = 1
        
        elif (self.CIR == "101"): #print
            self.index1, self.index2 = self.ret(self.MAR)
            print("INDEX", self.index1 + " INDEX2", self.index2)
            self.printC = 1
            
    def execute(self):
        if (self.loadGC == 1):
            if self.MDR == None:
                self.MDR = '{0:012b}'.format(int(self.dictG[self.target],2))
                print("LOAD GLOBAL MDR", self.MDR)
                print()
                self.loadGC = 0
                self.target = None
            else:
                self.AC = self.MDR
                self.MDR = '{0:012b}'.format(int(self.dictG[self.target],2))
                print("MOVE CONTENT TO AC")
                print("NEW MDR", self.MDR)
                print()
                self.loadGC = 0
                self.target = None
                
        elif (self.loadLC == 1):
            if self.MDR == None:
                self.MDR = '{0:012b}'.format(int(self.dictL[self.target],2))
                print("LOAD LOCAL MDR", self.MDR)
                print()
                self.loadLC = 0
                self.target = None
            else:
                self.AC = self.MDR
                self.MDR = '{0:012b}'.format(int(self.dictL[self.target],2))
                print("MOVE CONTENT TO AC")
                print("NEW MDR", self.MDR)
                print()
                self.loadLC = 0
                self.target = None

        elif (self.pushC == 1):
            if (self.pushS == '1'):
                print("PUSH", self.AC)
                self.dictL[self.target] = self.AC
                print("PUSH LOCAL", self.dictL[self.target])
            elif (self.pushS == '0'):
                print("PUSH", self.AC)
                self.dictG[self.target] = self.AC
                print("PUSH GLOBAL", self.dictG[self.target])
            
            self.AC = None
            self.pushC = 0
            self.target = None
            self.pushS = None
        
        elif (self.calC == 1):
            if self.AC != None:
                operation, result, overflow = self.ALU.operate(self.CIR, self.AC, self.MDR)
                time.sleep(.05)  #delay output
                print("Performed:", operation, self.AC, self.MDR + " Result:", result + " Overflow:", overflow) 
                print()
                self.AC = result
                self.calC = 0
           
        elif (self.retC == 1):
            if self.index2 == None and len(self.index1) > 7:
                print("RETURNING", self.index1 + "=> CONVERTING =>", str(int(self.index1, 2)))
                
            elif self.index2 == None and len(self.index1) == 7:
                print("RETURNING OPERAND AT ADDRESS", self.index1)
                print("OPERAND:", self.RAM[int(self.index1, 2)])
            
            elif self.index1 != None and self.index2 != None:
                print("RETURNING OPERAND AT ADDRESS", self.index1)
                print("OPERAND:", self.RAM[int(self.index1, 2)])
                
                print("RETURNING OPERAND AT ADDRESS", self.index2)
                print("OPERAND:", self.RAM[int(self.index2, 2)])
            
            elif self.index1 == None and self.index2 == None:
                print("RETURN CALCULATED VALUE", self.AC + "=>", str(int(self.AC), 2))
            
            elif self.index1 != None and self.index2 == "0":
                print("RETURN GLOBAL VAR AT INDEX", self.index)
                print("OPERAND", self.target + " VALUE:", str(int(self.dictG[self.target],2)))
                self.target = None
            
            elif self.index != None and self.index2 == "1":
                print("RETURN LOCAL VAR AT INDEX", self.index)
                print("OPERAND", self.target + " VALUE:", str(int(self.dictL[self.target],2)))
                self.target = None
            
            self.retC = 0
    
        elif (self.printC == 1):
            if self.index2 == None and len(self.index1) > 7:
                print("PRINTING", self.index1 + "=> CONVERTING =>", str(int(self.index1, 2)))
                
            elif self.index2 == None and len(self.index1) == 7:
                print("PRINTING OPERAND AT ADDRESS", self.index1)
                print("OPERAND:", self.RAM[int(self.index1, 2)])
            
            elif self.index1 != None and self.index2 != None:
                print("PRINTING OPERAND AT ADDRESS", self.index1)
                print("OPERAND:", self.RAM[int(self.index1, 2)])
                
                print("PRINTING OPERAND AT ADDRESS", self.index2)
                print("OPERAND:", self.RAM[int(self.index2, 2)])
            
            elif self.index1 == None and self.index2 == None:
                print("PRINT CALCULATED VALUE", self.AC + "=>", str(int(self.AC), 2))
            
            elif self.index1 != None and self.index2 == "0":
                print("PRINT GLOBAL VAR AT INDEX", self.index)
                print("OPERAND", self.target + " VALUE:", str(int(self.dictG[self.target],2)))
                self.target = None
            
            elif self.index != None and self.index2 == "1":
                print("PRINT LOCAL VAR AT INDEX", self.index)
                print("OPERAND", self.target + " VALUE:", str(int(self.dictL[self.target],2)))
                self.target = None
            
            self.printC = 0
        
        print(self.dictG, self.dictL)
        

In [266]:
class System():
    def __init__(self, RAM, dictG, dictL):
        self.dictG = dictG
        self.dictL = dictL
        self.RAM = RAM
        self.size = len(self.RAM)
        self.clock = time.time()     #start clock of the system
        self.AC = None   #Accumulator 
        self.N = 1
        self.firstUnit = PU(0, dictG, dictL, self.AC, self.RAM, self.size, self.N)  #give each unit a different PC
        self.secondUnit = PU(1, dictG, dictL, self.AC, self.RAM, self.size, self.N) #both share the AC, RAM
        self.counter = 0  #a counter for the loop
        #the PC of each Unit will increment by 2 because there are only two units
        #if the system was to have n units running at a time, then the PC of each will increment by n
        
    def delay(self):
        #delay method; delay by .25 sec
        time.sleep(.05)
    
    def run(self):
        """Run Method"""
        """ F D E
              F D E
            for N number of Units running, size is divided by n
            
        """
        while self.counter < (self.size // self.N):     
            self.firstUnit.fetch()
            self.delay()
            self.firstUnit.decode()
            self.firstUnit.execute()
            self.counter += 1
        #subtract current time with the time when the system starts
        print("\n" + str(round(time.time() - self.clock, 8)) + " s\n")    
    

In [267]:
sys = System(MC, dictG, dictL)
sys.run()

MAR: 0000000000001001 Instruction: 000 PC:  0
This is MDR 000000001001
{'a': '000011101001', 'z': '100000010001', 'b': '000001101111', 'c': '000001111010'} {'f': '000010000101', 'd': '000010001100'}
MAR: 0000000000000111 Instruction: 000 PC:  1
MOVE CONTENT TO AC 000000001001
NEW MDR 000000000111
{'a': '000011101001', 'z': '100000010001', 'b': '000001101111', 'c': '000001111010'} {'f': '000010000101', 'd': '000010001100'}
MAR: 0110000000000001 Instruction: 011 PC:  2
Performed: SUBTRACTION 000000001001 000000000111 Result: 000000010000 Overflow: False

{'a': '000011101001', 'z': '100000010001', 'b': '000001101111', 'c': '000001111010'} {'f': '000010000101', 'd': '000010001100'}
MAR: 0000000000011000 Instruction: 000 PC:  3
MOVE CONTENT TO AC 000000000111
NEW MDR 000000011000
{'a': '000011101001', 'z': '100000010001', 'b': '000001101111', 'c': '000001111010'} {'f': '000010000101', 'd': '000010001100'}
MAR: 1001000000000011 Instruction: 100 PC:  4
Performed: SUBTRACTION 000000000111 0000

INDEX 00001100100 INDEX2 None
PRINTING 00001100100=> CONVERTING => 100
{'a': '000011101001', 'z': '100000010001', 'b': '000001101111', 'c': '000001111010'} {'f': '000010000101', 'd': '000010001100'}
MAR: 1110000001100100 Instruction: 111 PC:  34
INDEX 00001100100 INDEX2 None
RETURNING 00001100100=> CONVERTING => 100
{'a': '000011101001', 'z': '100000010001', 'b': '000001101111', 'c': '000001111010'} {'f': '000010000101', 'd': '000010001100'}
MAR: 0010000000000000 Instruction: 001 PC:  35
LOAD GLOBAL VAR a
MOVE CONTENT TO AC
NEW MDR 000011101001

{'a': '000011101001', 'z': '100000010001', 'b': '000001101111', 'c': '000001111010'} {'f': '000010000101', 'd': '000010001100'}
MAR: 0000000001100100 Instruction: 000 PC:  36
MOVE CONTENT TO AC 000011101001
NEW MDR 000001100100
{'a': '000011101001', 'z': '100000010001', 'b': '000001101111', 'c': '000001111010'} {'f': '000010000101', 'd': '000010001100'}
MAR: 1000100011100100 Instruction: 100 PC:  37
Performed: SUBTRACTION 000011101001 000001100